In [62]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from kmodes.kprototypes import KPrototypes
from kmodes.kmodes import KModes
import matplotlib.pyplot as plt
import math


In [63]:
def cluster_data(data,data_label, n= 10):
    '''
    With the clean data and data mapping as inputs, this function does the following:
    1. Calculate the number of categorical and numerical features in the data.
    2. Choose the appropriate clustering algorithm among the three (K-means, K-mode, K-Prototype)
    3. Fit the data into the chosen clustering algorithm
    4. Choose the optimal number of clusters based on the calculated inertia
    5. Output the data with additional column of the cluster, and the centroids of each cluster
    
    '''
    
    data = data.dropna(axis='columns')
    #num_factors=data_label['Type'].value_counts()
    num_factors = data_label.Type.str.count("Numerical").sum()
    cat_factors = data_label.Type.str.count("Categorical").sum()
    cat_list = list(data_label.loc[data_label['Type'] == 'Categorical'].index.values)
    num_list = list(data_label.loc[data_label['Type'] == 'Numerical'].index.values)
    
    inertia = [] #where the data will be inputted
    k = range(1,n)
    data1 = data
    data  = data.values #transform it to matrix as an input to clustering algo
    if cat_factors == 0 and num_factors > 0:
        data_type = 1 #kmeans for numerical data
        for i in k: 
            kmeans = KMeans(n_clusters=i, random_state=0, init = 'k-means++').fit(data)
            inertia.append(kmeans.inertia_)

    elif cat_factors > 0 and num_factors == 0:
        data_type = 2 #kmode for categorical data
        for i in k: #kmodes
            kmode = KModes(n_clusters=i, init='Cao', verbose=0).fit(data)
            inertia.append(kmode.cost_)
    elif cat_factors > 0 and num_factors > 0:      
        data_type = 3
        for i in k: #kprototype for both types of data
            kproto = KPrototypes(n_clusters=i, init='Cao', verbose=0)
            clusters = kproto.fit_predict(data, categorical=cat_list)
            inertia.append(kproto.cost_)
    else:
        data_type = 0 # no data
        
    def calc_dist(x, y, a, b, c):
        d = abs((a * x + b * y + c))/ (math.sqrt(a*a + b*b))
        return d
    if data_type > 0:    
        a = inertia[0] - inertia[n-2]
        b = k[n-2] - k[0]
        c = k[0] * inertia[n-2] - k[n-2]*inertia[0]
        optimal_distance = []
        for i in range(n-2):
            optimal_distance.append(calc_dist(k[i], inertia[i], a, b, c))
        K = optimal_distance.index(max(optimal_distance)) + 1  #this outputs the optimal number of cluster

        if data_type == 1:
            kmeans = KMeans(n_clusters=K, random_state=0, init = 'k-means++').fit(data)
            labels = kmeans.predict(data)
            centroids = kmeans.cluster_centers_
            numerical = pd.DataFrame.from_records(centroids) #conversion of list of array to DF
            numerical.columns = list(data1.iloc[:, num_list].columns.values) #adding the factor to column header
            centroids = numerical #this is the highlighted characteristics per cluster
        elif data_type == 2:
            kmode = KModes(n_clusters=K, init='Cao', verbose=0).fit(data)
            centroids = kmode.cluster_centroids_
            labels = kmode.labels_
            categorical = pd.DataFrame.from_records(centroids) #conversion of list of array to DF
            categorical.columns = list(data1.iloc[:, cat_list].columns.values) #adding the factor to column header
            centroids = categorical #this is the highlighted characteristics per cluster
        elif data_type == 3:
            kproto = KPrototypes(n_clusters=K, init='Cao', verbose=0)
            clusters = kproto.fit_predict(data, categorical=cat_list)
            centroids = kproto.cluster_centroids_
            labels = kproto.labels_
            categorical = pd.DataFrame.from_records(centroids[1])#conversion of list of array to DF
            numerical = pd.DataFrame.from_records(centroids[0])#conversion of list of array to DF
            categorical.columns = list(data1.iloc[:, cat_list].columns.values) #adding the factor to column header
            numerical.columns = list(data1.iloc[:, num_list].columns.values) #adding the factor to column header
            centroids = numerical.join(categorical, how='outer') #this is the highlighted characteristics per cluster
        labels = pd.DataFrame(labels, columns = ['Cluster'])
        output = pd.concat([labels, data1], axis=1) # data with the cluster on the first column

        if data_type == 2 or data_type == 3:
            my_list = data_label.loc[data_label['Group'] > 0]["Field"].values # column header of the multi select cat variables
            my_list0 = data_label.loc[data_label['Group'] == 0]["Field"].values # column header of the non- multiselect cat variables
            my_list1 = data_label.loc[data_label['Group'] > 0]["Group"].values # group number per collective column header
            my_list2 = data_label.loc[data_label['Group'] > 0]["Factor"].values # collective column header of the multi select cat variables
            label = sorted(set(my_list2.tolist())) #list of collective column header
            with_g = centroids[my_list] #df for multi select cat variables
            with_g1 = centroids[my_list0] #df for non-multiselect cat variables
            with_g = with_g.apply(lambda x: x.astype(object).replace('1', x.name))
            with_g = with_g.replace('0', np.nan) #replaces 0 to null
            with_g = with_g.replace(0, np.nan) #replaces 0 to null
            with_g.columns = my_list1
            n = len(set(my_list1)) + 1
            mult = []
            for i in range(1,n):

                mult.append(pd.DataFrame(with_g[str(i)].apply(
                    lambda x: ','.join(x.dropna().astype(str)),
                    axis=1
                )))
            result_1 = pd.concat(mult, join='outer', axis=1).fillna('')
            result_1.columns = label
            centroids = pd.concat([with_g1, result_1], axis=1)
    else:
        centroids, output = 'no data','no data'
    return centroids, output
   

In [64]:
#Trying the function
sheet = 'numerical'
data = pd.read_excel('data.xlsx', sheet_name=sheet)
data_label = pd.read_excel('data.xlsx', sheet_name= sheet + '_mapping')
centroids, output = cluster_data(data, data_label)

In [65]:
#this outputs the mean for each cluster. Note index maps to its cluster number
centroids 

,Age,data1,data2,data3,data4
0,28.412806,84.984066,105.260844,6.020950,348.279729
1,28.218759,85.032057,105.832294,6.071238,448.896112
2,28.189389,84.773597,104.676126,5.981801,249.423812


In [66]:
#this outputs the data with additional column of the assigned cluster
output

,Cluster,Age,data1,data2,data3,data4
0,0,32,99,88,9,334
1,2,45,74,134,5,296
2,0,30,77,76,10,337
3,2,26,73,89,8,216
4,0,28,99,123,7,379
...,...,...,...,...,...,...
9995,1,30,84,115,5,423
9996,2,24,79,129,10,269
9997,1,35,97,97,9,456
9998,1,23,70,138,6,445
